<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/11_cnn_pytorch/09_multitask_fundamental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目的
マルチタスク学習の基礎を理解する．また，回帰タスクと識別タスクを同時に解くネットワークを実際に作成し，マルチタスク学習の性能を確認する．

# マルチタスク学習（MTL）
MTLとは，単一のニューラルネットワークで複数のタスクを解くことです．
例えば，[1]では歩行者の画像を入力して，歩行者の頭頂部と足先の座標，身体と顔の向き，性別，傘の有無を出力します．タスクごとに個別のネットワークを用いる場合と比べ，推定時の計算コストが少なく高速な推定ができるほか，各タスクがお互いの精度向上に貢献しあうことが知られています．

<table>
<tr>
<td><img src="http://www.mprg.cs.chubu.ac.jp/Tutorial/MPRG_Lecture/02_09/fukui1.png"></td>
<td><img src="http://www.mprg.cs.chubu.ac.jp/Tutorial/MPRG_Lecture/02_09/fukui2.png"></td>
</tr>
</table>

具体的には，ネットワークの出力ユニットを複数設け，それらの誤差関数を同時に最適化することで実現できます．出力ユニットは一般的に，[1]のようにネットワークの最終層で複数タスクの結果を同時に出力する方法や，ネットワークを途中で分岐させて各タスクの結果を出力する方法があります．基礎編では前者，応用編では後者のネットワークを取り扱います．誤差関数は，以下のように2つ以上のタスクの誤差を足します．このとき，誤差のオーダー（桁数）が大きく異る場合は，桁数の大きい誤差のタスクを重点的に学習してしまい，もう一方のタスクはあまり学習されないことがあります．これを解決するため，いずれかまたは全ての誤差に重み($\lambda$)をかけることがあります．

$$
L\left(\cdot\right)= \lambda_a L_{\mathrm{taskA}}\left(\hat{y}_{i}, y_{i}\right) + \lambda_b L_{\mathrm{taskB}}\left(\hat{z}_{i}, z_{i}\right)
$$

実は，近年多く提案されているモダンなディープニューラルネットワークのほとんどが，MTLの考え方に基づいています．例えば，物体検出を行うFaster R-CNN[2]は，物体の領域を検出するための誤差$L_{reg}$と，対象領域の物体クラスを識別するための誤差$L_{cls}$を同時に最適化しています．

# 準備
## Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います． GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

# 使用するデータセット
今回は，分類タスクと回帰タスクを同時に行うことができるMulti-Task Facial Landmark (MTFL) dataset[3]を用います．MTFLは，12995人の著名な人物の顔画像から構成されているデータセットです．教師ラベルとして，

- 各器官点の座標（両目，鼻，口角）
- 性別（男，女）
- 笑顔の判定（笑顔，真顔）
- 眼鏡の有無（あり，なし）
- 頭の向き（左横顔，左，正面，右，右横顔）

が含まれています．

<img src="http://www.mprg.cs.chubu.ac.jp/Tutorial/MPRG_Lecture/02_09/mtfl.jpg" width="50%">

# モジュールのインポートとGPUの確認
はじめに，必要なモジュールをインポートしたのち，GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です． `False`となっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
# モジュールのインポート
import os
from time import time
import numpy as np
import torch
import torch.nn as nn
import pandas

import torchvision

import torchsummary

import cv2
from google.colab.patches import cv2_imshow
import urllib.request
import zipfile

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

Use CUDA: True


# データセットの読み込み
データセットの読み込みは，他のチュートリアルと同様に行います．
まず，`MTFLdataset`というクラスを作成します． この際，`torch.utils.data.Dataset`クラスを継承します．

## 1. `__init__()`
`self.__init__(self, ...)`でMTFLデータセットのダウンロードと読み込みを行います． まず，`urllib`を用いてwebからデータをダウンロードします． その後，ダウンロードした圧縮ファイルを`zipfile`を用いて解凍します． ※ 解凍したファイルを左側の「ファイル」一覧から確認して見ましょう．

次に，用意するデータが学習用データ (`train`) か評価用データ (`test`) かを指定し，読み込むデータリスト (`training.txt`または`testing.txt`) を`self.data_list`に格納します．データリストのテキストファイルは下記のフォーマットになっています．

```
 #image path #x1...x5,y1..y5 #gender #smile #wearing glasses #head pose

--x1...x5,y1...y5: 各器官点の座標． 1: left eye, 2: right eye, 3: nose, 4: left mouth corner, 5: right mouth corner
--gender: 性別． 1: male, 2: female
--smile: 笑顔の判定． 1: smiling, 2: not smiling
--glasses: 眼鏡の有無． 1: wearing glasses, 2: not wearing glasses.
--head pose: 頭の向き． 1: left profile, 2: left, 3: frontal, 4: right, 5: right profile
```

このデータを格納するために，`pandas.read_csv()`を用います．この関数は本来，コンマで区切られたファイル（CSVファイル）を読み込むためのものですが，`sep=" "`を指定することで，半角スペースで区切られたファイルも読み込むことができます．また，1文字目に余分な半角スペースが入っているため`skipinitialspace=True`で無視するように指定し，最終行には無駄なデータ（半角スペースひとつだけ）があるため，`skipfooter=1`で最終行を無視するように指定します．


## 2. `__len__()`
`__len__(self)`ではデータセットのサンプル数を返すように定義します． 今回は，`self.data`に格納されている画像データの枚数を返す様に定義します． （学習用データでは10,000枚，評価用データは2,995枚）

## 3. `__getitem__()`
`__getitem__(self, item)`では，`item`で指定した番号のサンプルを読み込んで返すように定義を行います． まず，`item`番目の画像のパスを取得し，OpenCVの`imread`で読み込みます．pandasで読み込んだcsvは，`.iloc[]`を用いることでnumpyの配列のようにアクセスできます．また，パスの区切り文字はスラッシュとバックスラッシュが混ざっていることがあるため，ここではUnix（スラッシュ）に統一します．パスの文字列に`.replace('\\', '/')`をつけてバックスラッシュをスラッシュに置換します．

その後，画像を`40×40`にリサイズし，画像データの画素値を0~1の範囲の値になる様に正規化を行い，画像データの配列を`[channel, height, width]`となる様に配列操作を行います．

最後に，画像データと対応する教師ラベルを取得します．今回は鼻の座標と性別のデータのみ必要であるため，対象のデータを抽出してnumpyの配列にします．また，座標のデータは画像サイズで割り，0~1に正規化しておきます．




In [ ]:
class MTFLdataset(torch.utils.data.Dataset):
    base_folder = 'mtfl'
    url = "http://mmlab.ie.cuhk.edu.hk/projects/TCDCN/data/MTFL.zip"
    filename = "MTFL.zip"

    def __init__(self, root, train=True, download=True):
        super().__init__()

        self.root = root
        self.train = train
        self.download = download

        # MTFLデータのダウンロード
        if download and not(os.path.exists(os.path.join(self.root, self.filename))):
            print("Downloading dataset...")
            urllib.request.urlretrieve(self.url, os.path.join(self.root, self.filename))
            with zipfile.ZipFile(os.path.join(self.root, self.filename), 'r') as zipf:
                zipf.extractall(path=os.path.join(self.root, self.base_folder))

        # 学習，評価データの判定
        if self.train:
            self.data_list = pandas.read_csv(os.path.join(self.root, self.base_folder, "training.txt"), sep=" ", header=None, skipinitialspace=True, skipfooter=1, engine="python", 
                                          names=["#image path", "#x1","#x2","#x3","#x4","#x5","#y1","#y2","#y3"
                                                    ,"#y4","#y5","#gender"," #smile", "#wearing glasses", "#head pose"])
        else:
            self.data_list = pandas.read_csv(os.path.join(self.root, self.base_folder, "testing.txt"), sep=" ", header=None, skipinitialspace=True, skipfooter=1, engine="python", 
                                          names=["#image path", "#x1","#x2","#x3","#x4","#x5","#y1","#y2","#y3"
                                                    ,"#y4","#y5","#gender"," #smile", "#wearing glasses", "#head pose"])

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, item):
        img = cv2.imread(os.path.join(self.root, self.base_folder, self.data_list.iloc[item, 0]).replace('\\', '/'))
        original_height, original_width, _ = img.shape
        img = cv2.resize(img, (40, 40))
        #cv2_imshow(img)

        # データの正規化（0~255）
        img = img.astype(np.float32) / 255.

        # 画像の配列を入れ替え
        img = img.transpose(2, 0, 1)

        # 教師ラベルの読み込み・正規化
        target_csv = self.data_list.iloc[item]
        #target = np.asarray([target_csv["#x3"], target_csv["#y3"], target_csv["#gender"]], dtype=np.float32)
        #target[0] = target[0] / float(original_width)
        #target[1] = target[1] / float(original_height)
        target1 = np.asarray([target_csv["#x3"], target_csv["#y3"]], dtype=np.float32)
        target1[0] = target1[0] / float(original_width)
        target1[1] = target1[1] / float(original_height)
        target2 = np.asarray(target_csv["#gender"], dtype=np.long) - 1

        return img, target1, target2

上で定義したデータセットクラスを用いて，MTFLデータセットを読み込みます．

また，読み込んだデータセットクラスの情報を表示します． まず，各データセットが保有しているサンプル数を表示します． データセットクラスに`len()`を適用すると，上で定義した`__len__()`メソッドが呼ばれ，サンプル数を返します．

次に，`train_data`のとある1サンプルを読み込みます． `train_data[10]`とすることで，上で定義した`__getitem__()`メソッドが呼ばれ，引数の`item`に`10`が与えられ，10番目のサンプルを返します．

In [ ]:
train_data = MTFLdataset(root="./", train=True, download=True)
test_data = MTFLdataset(root="./", train=False, download=True)

# サンプル数の表示
print("# Training set:", len(train_data))
print("# Test set:", len(test_data))

# とあるサンプルの読み込み
img, label1, label2 = train_data[10]
print(img)
print(label1)
print(label2)

# Training set: 10000
# Test set: 2995
[[[0.6117647  0.61960787 0.63529414 ... 0.39607844 0.39215687 0.39215687]
  [0.60784316 0.6156863  0.6313726  ... 0.39607844 0.38039216 0.38039216]
  [0.6156863  0.62352943 0.6392157  ... 0.39607844 0.3882353  0.3882353 ]
  ...
  [0.16862746 0.05098039 0.08235294 ... 0.25490198 0.27058825 0.22352941]
  [0.0627451  0.03921569 0.05098039 ... 0.27450982 0.2627451  0.23529412]
  [0.27058825 0.09411765 0.04705882 ... 0.25882354 0.22352941 0.23137255]]

 [[0.61960787 0.6313726  0.64705884 ... 0.4117647  0.40784314 0.40784314]
  [0.61960787 0.627451   0.6431373  ... 0.4117647  0.39607844 0.39607844]
  [0.62352943 0.63529414 0.6509804  ... 0.4117647  0.40392157 0.40392157]
  ...
  [0.13725491 0.03529412 0.07450981 ... 0.1882353  0.20392157 0.14901961]
  [0.06666667 0.04313726 0.05882353 ... 0.2        0.19607843 0.16470589]
  [0.20392157 0.05098039 0.02352941 ... 0.19215687 0.15294118 0.16470589]]

 [[0.6117647  0.62352943 0.6392157  ... 0.39215687 0.3882

# ネットワークモデルの定義
今回用いる畳み込みニューラルネットワーク (CNN) を定義します．CNNは，[3]で用いられているものを参考に，畳み込み層4層，全結合層3層で構成します．

1層目の畳み込み層は入力チャンネル数が3，出力する特徴マップ数が16，畳み込むフィルタサイズが5x5です．2層目の畳み込み層は入力チャネル数が16．出力する特徴マップ数が48，畳み込むフィルタサイズは3x3です．同様に，3, 4層目の畳み込み層も準備します．また，各畳み込み層にはプーリングを適用しますが，プーリングは重みを持たず毎回同じ処理となるため，1つだけ定義します．1つ目の全結合層は入力ユニット数は2x2x64とし，出力は100としています．次の全結合層は入力が100，出力が24，最後の全結合層，つまり出力層は入力が24，出力が4です．出力ユニットは，0-1番目が鼻の座標 (x, y) ，2-3番目が性別 (male, female) に対応します．これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．`forward`関数の引数xは入力データです．それを`__init__`関数で定義した`conv1`に与え，その出力を`pool`に与えて，畳み込みとプーリングの処理を行います．プーリング処理結果はhとして出力し，`conv2`に与えられて次の層の処理を行います．これを繰り返して，すべての層が順番に処理されるように接続します．

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 48, kernel_size=3)
        self.conv3 = nn.Conv2d(48, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2 * 2 * 64,  100)
        self.fc2 = nn.Linear(100, 24)
        self.fc3 = nn.Linear(24, 4)
    
    def forward(self, x):
        h = self.pool(self.conv1(x))
        h = self.pool(self.conv2(h))
        h = self.pool(self.conv3(h))
        h = self.conv4(h)
        h = h.view(h.size()[0], -1)
        h = self.fc1(h)
        h = self.fc2(h)
        h = self.fc3(h)
        return h

# ネットワークの作成
上のプログラムで定義したネットワークを作成します．

CNNクラスを呼び出して，ネットワークモデルを定義します． また，GPUを使う場合 (`use_cuda == True`) には，ネットワークモデルをGPUメモリ上に配置します． これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します． また，学習率 (`lr`) を0.01，モーメンタム (`momentum`) を0.9として引数に与えます．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．畳み込みと全結合層には`Param #`にいくつかの値が存在しますが，これが重みパラメタの数となります．マックスプーリングは単に特徴マップのサイズを削減するだけなので，パラメタは存在しません．

In [ ]:
model = CNN()
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 40, 40))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 36, 36]           1,216
         MaxPool2d-2           [-1, 16, 18, 18]               0
            Conv2d-3           [-1, 48, 16, 16]           6,960
         MaxPool2d-4             [-1, 48, 8, 8]               0
            Conv2d-5             [-1, 64, 6, 6]          27,712
         MaxPool2d-6             [-1, 64, 3, 3]               0
            Conv2d-7             [-1, 64, 2, 2]          16,448
            Linear-8                  [-1, 100]          25,700
            Linear-9                   [-1, 24]           2,424
           Linear-10                    [-1, 4]             100
Total params: 80,560
Trainable params: 80,560
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.34
Params size (MB): 0.31
Estimated Tot

# 学習
読み込んだMTFLデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を10とします．

次にデータローダーを定義します．データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，`for`文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．

次に，誤差関数を設定します．今回はマルチタスク学習であるため，2つの誤差関数を定義します．回帰タスクを解くための二乗誤差を計算するための`MSELoss`を`criterion1`として，分類タスクを解くためのクロスエントロピー誤差を計算するための`CrossEntropyLoss`を`criterion2`として定義します．

そして，学習を開始します．誤差を二乗誤差とクロスエントロピー誤差と合計したものをそれぞれ表示するためにカウンターを初期化しておきます．

各更新において，学習用データと教師データをそれぞれ`image`と`label1`, `label2`とします．学習モデルに`image`をネットワークの出力`y`を取得します．`y`の中身は0-1番目が鼻の座標 (x, y) ，2-3番目が性別 (male, female) の確率を示しています．これらを対応する各教師データ (`label1`, `label2`) と対応する各誤差関数 (`criterion1`, `criterion2`) に入力して，誤差を計算します．誤差の逆伝播は1つの誤差値でしか行えないため，2つの誤差を単純に足します．はじめに述べたように，このとき誤差に重み値 (`lambda_1`, `lambda_2`)をかけます．今回はどちらも1として，重みを与えないようにします．足して1つにした誤差を`backward`関数で逆伝播し，ネットワークの更新を行います．認識精度も同時に計算して，`print`関数で学習経過における誤差や認識精度を表示します．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()
lambda_1 = 1.
lambda_2 = 1.
if use_cuda:
    criterion1.cuda()
    criterion2.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_reg_loss = 0.0
    sum_cls_loss = 0.0
    sum_all_loss = 0.0
    cls_count = 0
    reg_count = 0
    num_data = 0
    
    for image, label1, label2 in train_loader:
        if use_cuda:
            image = image.cuda()
            label1 = label1.cuda()
            label2 = label2.cuda()

        y = model(image)

        reg_loss = criterion1(y[:, :2], label1[:, :2])
        cls_loss = criterion2(y[:, 2:], label2)
        all_loss = lambda_1 * reg_loss + lambda_2 * cls_loss 

        model.zero_grad()
        all_loss.backward()
        optimizer.step()
        
        sum_reg_loss += reg_loss.item()
        sum_cls_loss += cls_loss.item()
        sum_all_loss += all_loss.item()
        norm = torch.sqrt(torch.pow(y[:, 0] - label1[:, 0], 2) + torch.pow(y[:, 1] - label1[:, 1], 2))
        reg_count += torch.sum(norm <= 0.1)
        cls_pred = torch.argmax(y[:, 2:], dim=1)
        cls_count += torch.sum(cls_pred == label2)
        num_data += len(image)
        
    print("epoch: {}, mean loss: {}, mean loss(reg): {}, mean loss(cls): {}, mean accuracy(reg): {}, mean accuracy(cls): {}, elapsed_time :{}".format(epoch,
                                                                                 sum_all_loss / n_iter,
                                                                                 sum_reg_loss / n_iter,
                                                                                 sum_cls_loss / n_iter,
                                                                                 reg_count.item() / float(num_data),
                                                                                 cls_count.item() / float(num_data),
                                                                                 time() - start))
print("Done.")
print("mean accuracy(reg): {} ({}/{})".format(reg_count.item() / float(num_data), reg_count.item(), num_data))
print("mean accuracy(cls): {} ({}/{})".format(cls_count.item() / float(num_data), cls_count.item(), num_data))

epoch: 1, mean loss: 0.6915680828094483, mean loss(reg): 0.008234763135015965, mean loss(cls): 0.6833333187103271, mean accuracy(reg): 0.6467, mean accuracy(cls): 0.5807, elapsed_time :15.013142585754395
epoch: 2, mean loss: 0.6847536476135254, mean loss(reg): 0.0035068423613905907, mean loss(cls): 0.6812468070983887, mean accuracy(reg): 0.7379, mean accuracy(cls): 0.5807, elapsed_time :29.599194288253784
epoch: 3, mean loss: 0.6740172958374023, mean loss(reg): 0.0035903671249747277, mean loss(cls): 0.6704269275665283, mean accuracy(reg): 0.7422, mean accuracy(cls): 0.5928, elapsed_time :44.36145043373108
epoch: 4, mean loss: 0.6276863939285279, mean loss(reg): 0.0029930346116423608, mean loss(cls): 0.6246933580398559, mean accuracy(reg): 0.7905, mean accuracy(cls): 0.6519, elapsed_time :59.15302515029907
epoch: 5, mean loss: 0.5452123704910279, mean loss(reg): 0.0023453194729983807, mean loss(cls): 0.542867052078247, mean accuracy(reg): 0.8655, mean accuracy(cls): 0.7274, elapsed_time

# テスト
学習したネットワークモデルを用いて評価（テスト）を行います．テストは100枚ずつ行うため，`batch_size`は100とします．データをシャッフルする必要はないため，`shuffle=False`とします．学習時と同様に，`for`文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．

すべての画像（2,995枚）でテストが終わったら，最終的な精度を表示します．回帰タスクが75〜80%，分類タスクが60〜70%程度の性能になるはずです．


In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
cls_count = 0
reg_count = 0
num_data = 0
with torch.no_grad():
    for image, label1, label2 in test_loader:

        if use_cuda:
            image = image.cuda()
            label1 = label1.cuda()
            label2 = label2.cuda()
            
        y = model(image)

        norm = torch.sqrt(torch.pow(y[:, 0] - label1[:, 0], 2) + torch.pow(y[:, 1] - label1[:, 1], 2))
        reg_count += torch.sum(norm <= 0.1)
        cls_pred = torch.argmax(y[:, 2:], dim=1)
        cls_count += torch.sum(cls_pred == label2)
        num_data += len(image)

print("test accuracy(reg): {}({}/{})".format(reg_count.item() / float(num_data), reg_count.item(), num_data))
print("test accuracy(cls): {}({}/{})".format(cls_count.item() / float(num_data), cls_count.item(), num_data))

test accuracy(reg): 0.7549248747913189(2261/2995)
test accuracy(cls): 0.66110183639399(1980/2995)


# マルチタスクにしない場合との比較
マルチタスク学習を行わず，各タスクをそれぞれ別のネットワークで学習した場合と比較します．ネットワークは最終層のユニットを2つにして，`single_CNN`とします．そして，ネットワークとオプティマイザを個別に定義し，以降すべての学習処理を2つの異なるネットワークで同時に進めます．学習が終了したら，そのまま評価を行います．

In [ ]:
class single_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 48, kernel_size=3)
        self.conv3 = nn.Conv2d(48, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2 * 2 * 64,  100)
        self.fc2 = nn.Linear(100, 24)
        self.fc3 = nn.Linear(24, 2)
    
    def forward(self, x):
        h = self.pool(self.conv1(x))
        h = self.pool(self.conv2(h))
        h = self.pool(self.conv3(h))
        h = self.conv4(h)
        h = h.view(h.size()[0], -1)
        h = self.fc1(h)
        h = self.fc2(h)
        h = self.fc3(h)
        return h

model_reg = single_CNN()
model_cls = single_CNN()
if use_cuda:
    model_reg.cuda()
    model_cls.cuda()

optimizer_reg = torch.optim.SGD(model_reg.parameters(), lr=0.01, momentum=0.9)
optimizer_cls = torch.optim.SGD(model_cls.parameters(), lr=0.01, momentum=0.9)


## TRAIN

# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion_reg = nn.MSELoss()
criterion_cls = nn.CrossEntropyLoss()
if use_cuda:
    criterion_reg.cuda()
    criterion_cls.cuda()

# ネットワークを学習モードへ変更
model_reg.train()
model_cls.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_reg_loss = 0.0
    sum_cls_loss = 0.0
    cls_count = 0
    reg_count = 0
    num_data = 0
    
    for image, label1, label2 in train_loader:
        if use_cuda:
            image = image.cuda()
            label1 = label1.cuda()
            label2 = label2.cuda()

        y_reg = model_reg(image)
        y_cls = model_cls(image)

        reg_loss = criterion_reg(y_reg, label1)
        cls_loss = criterion_cls(y_cls, label2)

        model_reg.zero_grad()
        model_cls.zero_grad()
        reg_loss.backward()
        cls_loss.backward()
        optimizer_reg.step()
        optimizer_cls.step()
        
        sum_reg_loss += reg_loss.item()
        sum_cls_loss += cls_loss.item()
        norm = torch.sqrt(torch.pow(y_reg[:, 0] - label1[:, 0], 2) + torch.pow(y_reg[:, 1] - label1[:, 1], 2))
        reg_count += torch.sum(norm <= 0.1)
        cls_pred = torch.argmax(y_cls, dim=1)
        cls_count += torch.sum(cls_pred == label2)
        num_data += len(image)
        
    print("epoch: {}, mean loss(reg): {}, mean loss(cls): {}, mean accuracy(reg): {}, mean accuracy(cls): {}, elapsed_time :{}".format(epoch,
                                                                                 sum_reg_loss / n_iter,
                                                                                 sum_cls_loss / n_iter,
                                                                                 reg_count.item() / float(num_data),
                                                                                 cls_count.item() / float(num_data),
                                                                                 time() - start))
print("Done.")
print("mean accuracy(reg): {} ({}/{})".format(reg_count.item() / float(num_data), reg_count.item(), num_data))
print("mean accuracy(cls): {} ({}/{})".format(cls_count.item() / float(num_data), cls_count.item(), num_data))


## TEST

# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_reg.eval()
model_cls.eval()

# 評価の実行
cls_count = 0
reg_count = 0
num_data = 0
with torch.no_grad():
    for image, label1, label2 in test_loader:

        if use_cuda:
            image = image.cuda()
            label1 = label1.cuda()
            label2 = label2.cuda()
            
        y_reg = model_reg(image)
        y_cls = model_cls(image)

        norm = torch.sqrt(torch.pow(y_reg[:, 0] - label1[:, 0], 2) + torch.pow(y_reg[:, 1] - label1[:, 1], 2))
        reg_count += torch.sum(norm <= 0.1)
        cls_pred = torch.argmax(y_cls, dim=1)
        cls_count += torch.sum(cls_pred == label2)
        num_data += len(image)

print("test accuracy(reg): {}({}/{})".format(reg_count.item() / float(num_data), reg_count.item(), num_data))
print("test accuracy(cls): {}({}/{})".format(cls_count.item() / float(num_data), cls_count.item(), num_data))


epoch: 1, mean loss(reg): 0.014211838310956956, mean loss(cls): 0.6854059036254883, mean accuracy(reg): 0.6256, mean accuracy(cls): 0.5717, elapsed_time :15.328052997589111
epoch: 2, mean loss(reg): 0.003456924858689308, mean loss(cls): 0.6822478080749512, mean accuracy(reg): 0.7416, mean accuracy(cls): 0.5807, elapsed_time :30.6910502910614
epoch: 3, mean loss(reg): 0.0034310488805174826, mean loss(cls): 0.6771477939605713, mean accuracy(reg): 0.7431, mean accuracy(cls): 0.5813, elapsed_time :45.93756127357483
epoch: 4, mean loss(reg): 0.003406249889731407, mean loss(cls): 0.6354801582336426, mean accuracy(reg): 0.745, mean accuracy(cls): 0.6417, elapsed_time :61.151575803756714
epoch: 5, mean loss(reg): 0.00337755616158247, mean loss(cls): 0.5532261325836182, mean accuracy(reg): 0.7461, mean accuracy(cls): 0.7159, elapsed_time :76.2879695892334
epoch: 6, mean loss(reg): 0.0033528373032808306, mean loss(cls): 0.512042798614502, mean accuracy(reg): 0.7493, mean accuracy(cls): 0.7492, e

# 課題
1. 鼻の位置・性別以外でも学習・テストしてみよう．
2. 誤差に重みをかけて，精度が上がるのか確かめてみよう．

## ヒント
1. `MTFLdataset`の，以下の部分を変更します．
```
        target1 = np.asarray([target_csv["#x3"], target_csv["#y3"]], dtype=np.float32)
        target2 = np.asarray(target_csv["#gender"], dtype=np.long) - 1
```

2. 通常，「誤差が早く収束するタスク」「誤差の値が小さいタスク」の誤差に少数（`0.001`など）の重みをかけて調整します．ただし，例外も多いため，実験的に見つける事がほとんどです．

# 参考文献
- [1] H. Fukui, T. Yamashita, Y. Yamauchi, H. Fujiyoshi, and H. Murase, "Training of CNN with Heterogeneous Learning for Multiple Pedestrian Attributes Recognition Using Rarity Rate", IEICE TRANSACTIONS on Information and Systems, Vol.E101-D, No.5, pp.1222-1231, 2018.
- [2] S. Ren, K. He, R. Girshick, and J. Sun, "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks". Advances in Neural Information Processing Systems, pp. 91-99, 2015.
- [3] Z. Zhang, P. Luo, C. C. Loy, and X. Tang, "Facial Landmark Detection by Deep Multi-task Learning", in Proceedings of European Conference on Computer Vision, 2014.